# ClusterAI 2024

# Ciencia de Datos - Ingeniería Industrial - UTN BA

# clase_10: Practica Redes Neuronales

### Elaborado por: Aguirre Nicolas

# IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
pd.set_option('display.float_format', lambda x: '%.1d' % x) # Para acotar los decimales en pandas
np.set_printoptions(precision=5, suppress=True)

# IRIS DATASET

En esta primera ejercitacion vamos a retomar el dataset Iris (visto en la Clase 4)


El conjunto de datos contiene 50 muestras de cada una de tres especies de Iris (Iris setosa, Iris virginica e Iris versicolor). Se midió cuatro rasgos de cada muestra: lo largo y lo ancho del sépalos y pétalos, en centímetros. Basado en la combinación de estos cuatro rasgos.

https://es.wikipedia.org/wiki/Iris_flor_conjunto_de_datos

Los datos son:

| Columna | Descripcion |
| --- | --- |
| ID | Unique ID |
| SepalLengthCm | Length of the sepal (cm) |
| SepalWidthCm | Width of the sepal (cm) |
| PetalLengthCm | Length of the petal (cm) |
| PetalWidthCm | Width of the petal (cm) |
| Species | name |


In [ ]:
# Primero cargamos los datos que ya vienen incluidos en la libreria sk-learn.
from sklearn.datasets import load_iris
iris = load_iris()

X = iris.data
Y = iris.target

In [ ]:
n_features = np.shape(X)[0]
n_samples = np.shape(X)[1]
n_classes = np.unique(Y)
print(f'Features: ',n_samples)
print(f'Samples: ',n_features)
print(f'Classes: ',n_classes)

In [ ]:
# Veamos la primera sample
print(f'X: {X[0]}')
print(f'Y: {Y[0]}')

## SPLIT

In [ ]:
# Separamos train y test set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
# Separamos train y validation set
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

## SCALING

In [ ]:
# Noralizamos
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train_norm = scaler.transform(x_train)
x_val_norm = scaler.transform(x_val)
x_test_norm = scaler.transform(x_test)

# Modelo

In [ ]:
# Pytorch
import torch
print('Version de Pytorch: ',torch.__version__)
import torch.nn as nn
from torch.utils.data import TensorDataset,Dataset, DataLoader
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Modelo

Al igual que TensorFlow-Keras, PyTorch es una libreria para codear modelos de NN y tambien tiene su modelo Sequential.

Pero a diferencia de TF-Keras, PyTorch no tiene nativamente las funciones **fit**, **evaluate** y **predict**.

Por lo cual somos nosotros quienes vamos a tener codear el entrenamiento ...

El tipo de arquitectura que vamos a utilizar es una NN *fully-conected*. Para esto utilizaremos la funcion *nn.Sequential()* a la cual le pasaremos los distintos componentes de la red.

Cuando usamos una arquitectura secuencial, vamos construyendo el **foreward pass** de la red de manera tal que la red utiliza la salida de la capa inmediatamente anterior $z^{l-1}$ en la capa siguiente z^{l}, y asi sucecivamente hasta llegar al ultimo elemento de la red con el cual generamos el output.

$$
z^{l} = \sigma~(Wz^{l-1}+b)
$$

Puntualmente, en este ejemplo utilizaremos unicamente dos componentes.

* [**nn.Linear**](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) : Para aplicar la transformacion lineal $\delta = Wx + b$,
donde $\delta$ es llamada tambien *pre-activacion* de la neurona.

* [**nn.Sigmoid**](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html) : Para aplicar la funcion $\sigma(\delta) = \frac{1}{1 + \exp(-\delta)}$

En donde $W$ son los pesos (weights) que aprendera la red.


Cada una de las capas de nn.Linear se construye dandole la informacion de la cantidad de features de entrada y de salida. Como en nuestro ejemplo tenemos 3 clases de flores, la ultima capa debera tener una salida de dimension = 3.

<img src="NN.jpg" width="1200">

(Nota: ver `NN.jpg`)

In [ ]:
input_features = 4
layers_features = 4
output_dim = 3
model = nn.Sequential(
          nn.Linear(input_features,layers_features),
          nn.Sigmoid(),
          nn.Linear(layers_features,layers_features),
          nn.Sigmoid(),
          nn.Linear(layers_features,layers_features),
          nn.Sigmoid(),
          nn.Linear(layers_features,output_dim)
          )

In [ ]:
model

## Ejemplo

In [ ]:
# Imaginemos ahora 2 muestras
y_true_example = torch.tensor([0, 2], dtype=torch.long)

# Para las cuales la red genera la siguiente salida del 'softmax'
ejemplo_1 = torch.tensor([[3.5, 5, -3]])
ejemplo_2 = torch.tensor([[0.25, 0.25, 0.5]])
softmax_1 = torch.nn.functional.softmax(ejemplo_1, dim=1)
softmax_2 = torch.nn.functional.softmax(ejemplo_2, dim=1)
print(softmax_1.numpy())
print(softmax_2.numpy())
softmax =  torch.cat((softmax_1, softmax_2), dim=0)
print("Softmax")
softmax

### Softmax
$$
\hat y_j = \mathrm{Softmax} (\text{output}_j) = \frac{\exp(\text{output}_j)}{\sum_k \exp(\text{output}_k)}\\
\\[5ex]
$$


$$
Ejemplo:\\
\text{output}_0 = 3.5\\
\text{output}_1 = 5\\
\text{output}_2 = -3\\
$$


\begin{align}
\hat y_0 =&~\mathrm{Softmax} (\text{output}_0) &=~& \frac{\exp(\text{output}_0)}{\exp(\text{output}_0)+\exp(\text{output}_1)+\exp(\text{output}_2)}\\[5ex]
\hat y_0 =&                         &=~& \frac{\exp(3.5)}{\exp(3.5)+\exp(5)+\exp(-3)}\\[5ex]
\hat y_0 =&                         &=~& \frac{33.11}{33.11+148.41+0.04}\\[5ex]
\hat y_0 =&                         &=~& 0.182
\end{align}



In [ ]:
# BIEN O MAL CLASIFICADO?
y_pred_example = torch.cat((ejemplo_1, ejemplo_2), dim=0)
y_pred_example

### Cross-Entropy

$H(P, Q) \stackrel{\mathrm{def}}{=} \sum_j - P(j) \log Q(j)$

$\mathcal{L}_{CE}( y, \hat{y}) = - \sum_{j=1}^{q} y_j \log \hat{y}_j$

In [ ]:
# Veamos como varia la loss en base a las predicciones de la red
loss_func = nn.CrossEntropyLoss(reduction='none')
# El 'none' en la reduccion, es para obtener la loss individualmente.
# Tiene otros usos, pero escapan al curso.

# Ahora veamos como daria la loss en cada caso:
loss_func(y_pred_example, y_true_example).numpy()

[Documentacion](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

Confirmemos que lo que estamos haciendo es efecticamente la cross-entropy ...

In [ ]:
print(-np.log(softmax[0][0]))

## Optimizador y Loss function

In [ ]:
# Learning rate y Optimizador
lr = 0.05
optimizador = torch.optim.SGD(model.parameters(),lr=lr)
# Funcion de penalizacion
loss_func = nn.CrossEntropyLoss()

# Tensor Dataset

En general, cuando entrenamos NN los datos deben estar contenidos en tensores.
Un tensor es una generalización de los vectores y las matrices y se entiende fácilmente como una matriz multidimensional.

Ejemplo:

1. Un vector es un 1D tensor de dimension $1x5$

2. Una matriz es un 2D tensor de dimension $2x5$

En particular, dentro del área de Deep Learning se llama Tensor a aquellas estructura de datos que son capaces de realizar operaciones en **paralelo** dentro de las GPUs, aumentando significativamente nuestra capacidad/velocidad de computo.

Por ultimo deberemos definir el generador de datos para entrenar en batches. Estos objetos se llaman Dataloaders.

Para crearlo le pasamos nuestro dataset y el tamaño de nuestro batch.

In [ ]:
# Batch size
bs = 8

# Pasamos nuestro numpy arrays, a Tensor
x_train_norm_t, y_train_t = torch.Tensor(x_train_norm), torch.LongTensor(y_train) # Los Long Tensor son int
x_val_norm_t, y_val_t = torch.Tensor(x_val_norm), torch.LongTensor(y_val)
x_test_norm_t, y_test_t = torch.Tensor(x_test_norm), torch.LongTensor(y_test)

# Creamos los Dataset
train_ds = TensorDataset(x_train_norm_t,y_train_t)
val_ds = TensorDataset(x_val_norm_t,y_val_t)
test_ds = TensorDataset(x_test_norm_t,y_test_t)

# Creamos los Dataloader
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)

## Training

En este punto ya contamos con todas las "herramientras" que necesitamos para entrenar un modelo de NN.

* Datos
* Optimizador
* Funcion de Penalizacion
* Red Neuronal

Vamos a ver como es que esto elementos "interactuan" para actualizar los parametros/*weights* de la Red Neuronal ... es decir ... para entrenarla.

NN_Flow.svg

<img src="NN_Flow.jpg" width="1200">

In [ ]:
# Cantidad de Epochs
n_epochs = 900

# Generamos un diccionario donde vamos a guaradr historial de entrenamiento
training = {'train':{'loss':[],
                    'acc':[]},
           'val':{'loss':[],
                  'acc':[]}
           }

In [ ]:
################################################
# TRAINING: Epochs
################################################
for epoch in range(n_epochs):
  # Ahora nosotros vamos a tener que definir estas variables ...
  train_loss = 0
  cls_correctas_train = 0
  train_acc = 0
  ################################################
  # TRAINING: Batch
  ################################################
  for i , (x_batch, y_batch) in enumerate(train_dl):
    # Limpiamos todos los gradientes cargados en el optimizador
    optimizador.zero_grad()
    # Con un x_batch generamos una prediccion
    y_pred = model(x_batch)
    # La clase predicha será el índice de máximo valor luego del
    # softmax (integrado en CrossEntropyLoss)
    _, predicted = torch.max(y_pred.data, 1)
    # Calculamos la loss
    batch_loss = loss_func(y_pred,y_batch)
    # Calculamos el gradiente de la loss
    # Aca es donde sucede el back-propagation a.k.a. "la magia"
    batch_loss.backward()
    # Ajustamos los parametros del modelo con el optimizador
    optimizador.step()
    # Acumulamos la loss
    train_loss += batch_loss.item()
    # Sumamos la cantidad de clases correctas en el batch
    correct_i = (predicted == y_batch).sum().item()
    # Acumulamos corrects
    cls_correctas_train += correct_i

  ################################################
  # TRAINING: Validation
  ################################################
  val_loss = 0
  cls_correctas_val = 0
  val_acc = 0
  with torch.torch.inference_mode():
    for x_batch, y_batch in val_dl:
      y_pred = model(x_batch)
      _, predicted = torch.max(y_pred.data, 1)
      batch_loss = loss_func(y_pred,y_batch)
      val_loss += batch_loss.item()
      correct_i = (predicted == y_batch).sum().item()
      cls_correctas_val += correct_i

  # Calculamos el accuracy
  train_acc = (cls_correctas_train / len(train_ds))*100    # Calculamos el accuracy
  val_acc = (cls_correctas_val / len(val_ds))*100
  mean_train_loss = train_loss/len(train_ds)
  mean_val_loss = val_loss/len(val_ds)

  # Imprimimos en pantalla
  print('Epoch: {} T_Loss: {:.4f} T_Acc: {:.2f}%  V_Loss: {:.4f} V_Acc: {:.2f} %'.format(
      epoch,mean_train_loss,train_acc,mean_val_loss,val_acc))

  # Guardamos en el historial de entrenamiento ...
  training['train']['loss'].append(mean_train_loss)
  training['train']['acc'].append(train_acc)
  training['val']['loss'].append(mean_val_loss)
  training['val']['acc'].append(val_acc)

In [ ]:
#Loss
train_loss_h = training['train']['loss']
val_loss_h = training['val']['loss']
# Acc
train_acc_h = training['train']['acc']
val_acc_h = training['val']['acc']
lepochs = range(1, len(train_loss_h) + 1)

fig,(axs1,axs2) = plt.subplots(1,2,figsize=(16,6))
axs1.plot(lepochs, train_loss_h, 'b', label='Train loss')
axs1.plot(lepochs, val_loss_h, 'r', label='Val loss')
axs1.set_title('Training and validation Loss',fontsize=20)
axs1.set_ylabel('Loss',fontsize=16)
axs1.legend(fontsize=16)
axs2.plot(lepochs, train_acc_h, 'b', label='Train Accuracy')
axs2.plot(lepochs, val_acc_h, 'r', label='Validation Accuracy')
axs2.set_title('Training and validation Accuracy',fontsize=20)
axs2.set_ylabel('Accuracy [%]',fontsize=16)
axs2.legend(fontsize=16)
plt.show()

# **PREGUNTAS**:
```
1) Es correcto que nuestra loss de train y de validacion disminuyan, y sin embargo el accuracy se mantenga igual/baje ? Por que ?

2) Que cambios podriamos hacer para intentar solucionar el entrenamiento con la sigmoid function?
```



# DESAFIO

Generar una funcion llamada **`func_train()`** y **`func_val()`** para reemplezar en el for loop de entrenamiento.

Luego, generar una fucion llamada **`func_fit()`** la cual reemplace todo el entrenamiento y validacion (debe contener dentro de si **`func_train()`** y **`func_val()`** ).

Ayuda:

`func_fit()` debe recibir como argumentos (inputs)

* el `modelo`, `optimizador`, `loss function`, `dataloaders`, `datasets` y la cantidad de `epochs` a entrenar

* el return de la funcion es el `modelo` ya entrenado y el `historial` de entrenamiento.



In [ ]:
def func_train(model, train_dl,train_ds, loss_func, optimizador):
  train_loss = 0
  cls_correctas_train = 0
  train_acc = 0
  ################################################
  # TRAINING
  ################################################
  for i , (x_batch, y_batch) in enumerate(train_dl):
    # Limpiamos todos los gradientes cargados en el optimizador
    optimizador.zero_grad()
    # Con un x_batch generamos una prediccion
    y_pred = model(x_batch)
    # La clase predicha será el índice de máximo valor luego del
    # softmax (integrado en CrossEntropyLoss)
    _, predicted = torch.max(y_pred.data, 1)
    # Calculamos la loss
    batch_loss = loss_func(y_pred,y_batch)
    # Calculamos el gradiente de la loss
    # Aca es donde sucede el back-propagation a.k.a. "la magia"
    batch_loss.backward()
    # Ajustamos los parametros del modelo con el optimizador
    optimizador.step()
    # Acumulamos la loss
    train_loss += batch_loss.item()
    # Sumamos la cantidad de clases correctas en el batch
    correct_i = (predicted == y_batch).sum().item()
    # Acumulamos corrects
    cls_correctas_train += correct_i
  # Calculamos el accuracy
  train_acc = (cls_correctas_train / len(train_ds))*100    # Calculamos el accuracy
  mean_train_loss = train_loss/len(train_ds)

  return (model, mean_train_loss,  cls_correctas_train,  train_acc)

In [ ]:
def func_validation(model, val_dl,val_ds,loss_func):
  val_loss = 0
  cls_correctas_val = 0
  val_acc = 0
  ################################################
  # VALIDATION
  ################################################
  with torch.torch.inference_mode():
    for x_batch, y_batch in val_dl:
      y_pred = model(x_batch)
      _, predicted = torch.max(y_pred.data, 1)
      batch_loss = loss_func(y_pred,y_batch)
      val_loss += batch_loss.item()
      correct_i = (predicted == y_batch).sum().item()
      cls_correctas_val += correct_i
  # Calculamos el accuracy
  val_acc = (cls_correctas_val / len(val_ds))*100
  mean_val_loss = val_loss/len(val_ds)
  return (mean_val_loss, cls_correctas_val, val_acc)

In [ ]:
def func_fit(n_epochs,model,
             train_dl,train_ds,val_dl,val_ds,
             loss_func,optimizador):

  # Generamos un diccionario donde vamos a guaradr historial de entrenamiento
  training = {'train':{'loss':[],
                      'acc':[]},
            'val':{'loss':[],
                    'acc':[]}
            }
  # For loop epochs
  for epoch in range(n_epochs):
    model,mean_train_loss, cls_correctas_train, train_acc =  func_train(model, train_dl,train_ds, loss_func, optimizador)
    mean_val_loss, cls_correctas_val, val_acc = func_validation(model, val_dl,val_ds,loss_func)
    # Imprimimos en pantalla
    print('Epoch: {} T_Loss: {:.4f} T_Acc: {:.2f}%  V_Loss: {:.4f} V_Acc: {:.2f} %'.format(
        epoch,mean_train_loss,train_acc,mean_val_loss,val_acc))

    # Guardamos en el historial de entrenamiento ...
    training['train']['loss'].append(mean_train_loss)
    training['train']['acc'].append(train_acc)
    training['val']['loss'].append(mean_val_loss)
    training['val']['acc'].append(val_acc)
  return (model,training)

In [ ]:
# Cantidad de Epochs a entrenar
n_epochs = 800

model,training = func_fit( n_epochs,model,train_dl,train_ds,val_dl,val_ds,loss_func,optimizador)

# Creacion del Modelos
## (Avanzado y No Obligatorio para el curso !)
En la siguiente seccion veremos como se definen los modelos de NN de un modo mas avanzado.


In [ ]:
class Red_Neuronal_Personalizada(nn.Module):
    def __init__(self,input_features,layers_features,output_dim):
        super(Red_Neuronal_Personalizada, self).__init__()
        # Primera Capa
        self.layer_1 = nn.Linear(input_features,layers_features)
        self.act_1 = nn.Sigmoid()
        # Segunda Capa
        self.layer_2 = nn.Linear(layers_features,layers_features)
        self.act_2 = nn.Sigmoid()
        # Tercera Capa
        self.layer_3 = nn.Linear(layers_features,layers_features)
        self.act_3 = nn.Sigmoid()
        # Ultima Capa
        self.layer_4 = nn.Linear(layers_features,output_dim)

    def forward(self, x):
        z = self.act_1(self.layer_1(x))
        z = self.act_2(self.layer_2(z))
        z = self.act_3(self.layer_3(z))
        output = self.layer_4(z)
        return output

Esto a primera vista puede ser un poco abrumador, pero vayamos viendo de a poco que es lo que hace cada parte del codigo.



```python
class Red_Neuronal_Personalizada(nn.Module):
```
En esta parte definimos una clase que llamamos **Red_Neuronal_Personalizada**, que nos permitira luego crear unaobjetos/instancias de dicha clase.

```python
modelo = Red_Neuronal_Personalizada()
```
Es similiar a cuando definimos una funcion, solo que es con objetos de PyThon.

```python
def func_propia():

class clase_propia():
```

Con respecto a la siguiente parte del código
```python
    def __init__(self):
        super(Red_Neuronal_Personalizada, self).__init__()
```        

Simplemente le estamos diciendo de que partes/atributos estará compuesta nuestra clase/modelo cuando lo creemos.

La funcion __init__ es una funcion interna de Python que se ejecuta cuando creamos una instancia de nuestro objeto. Es decir, cuando ejecutemos

```python
modelo = Red_Neuronal_Personalizada()
```

---


Hasta este punto las partes de nuestro modelo aun no las hemos definido, asi que es lo que haremos en la siguientes lineas:

```python
class Red_Neuronal_Personalizada(nn.Module):
    def __init__(self):
        super(Red_Neuronal_Personalizada, self).__init__()
        # Primera Capa
        self.layer_1 = nn.Linear(input_features,layers_features)
        self.act_1 = nn.Sigmoid()
        # Segunda Capa
        self.layer_2 = nn.Linear(layers_features,layers_features)
        self.act_2 = nn.Sigmoid()
        # Tercera Capa
        self.layer_3 = nn.Linear(layers_features,layers_features)
        self.act_3 = nn.Sigmoid()
        # Ultima Capa
        self.layer_4 = n.Linear(layers_features,output_dim)
```

* `self.` : Siempre que definimos las partes (formalmente llamado **atributo**) de las que estara compuesta un objeto en python, se **TIENE** que anteponer la palabra `self.nombre_parte`. De esta manera, cuando querramos que nuestro objeto utilice de alguna manera dicha parte, lo indicaremos de la misma manera: `self.nombre_parte()`

En este ejemplo concreto, nuestro modelo / objeto tendra 7 componentes:

* Layers 1, 2, 3 y 4
* Activaciones 1, 2 y 3

Todas estas componentes son objetos que importamos de la libreria PyTorch.

---


Veamos la siguiente linea de nuestro codigo:
```python
def forward(self, x):
```  

En esta seccion definiremos el forward pass nosotros mismo. Como vemos, forward es una funcion que pertenecera a nuestro objeto y se utilizara cuando durante el entrenamiento pasemos la siguiente info.

```python
prediccion = model (x_batch)
```
`x_batch` en esta parte hace referencia a la **x** en:

```python
def forward(self, x):
```

y `self`, se lo pasaremos SIEMPRE que definimos una funcion en nuestro objeto que necesitemos que tenga acceso a los atributos que definimos en la seccion `____init ____`

Bueno, sabiendo ya lo que haremos en esta funcion, pasemos a la siguiente parte del codigo ...


```python
    def forward(self, x):
        z = self.act_1(self.layer_1(x))
        z = self.act_2(self.layer_2(z))
        z = self.act_3(self.layer_3(z))
        output = self.layer_4(z)
        return output
```        

En cada una de las lineas, obtenemos la salida de cada una de las capas. Fijense en particular que llamamos a las partes/atributos con **`self.`**

En particular :
```python
z = self.act_1(self.layer_1(x))
```

Equivale a:       $z = \sigma ( W \cdot X + b)$

---

Con esto ultimo quedo definida la arquitectura de nuestra red!

Vamos ahora a crear una instancia!

Si observamos nuevamente en la funcion que genera los objetos




```python
def __init__(self,input_features,layers_features,output_dim):
```

Le pasamos los argumentos:

* input_features
* layers_features
* output_dim

Asi que vamos a definirlos para crear una intancia llamada **modelo_personalizado** de nuestra clase **Red_Neuronal_Personalizada**


In [ ]:
input_features = 4
layers_features = 4
output_dim = 3
modelo_personalizado = Red_Neuronal_Personalizada(input_features,layers_features,output_dim)

In [ ]:
type(modelo_personalizado)

In [ ]:
modelo_personalizado

Listo!

Ahora entendemos que es lo que suecede internamente en **`nn.Sequential()`**, con la gran diferencia que ahora tenemos plena capacidad para definir las partes de nuestra red, su interaccion y el **forward pass**.


